In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split, LeaveOneOut, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
df = pd.read_csv('IphoneData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
X = newdf.to_numpy()
y = df['CO2E'].to_numpy()

C:\Users\emdua\AppData\Local\Temp\ipykernel_20668\1887309404.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [106]:
searchSpace = {
    "n_estimators": [75],
    "max_depth": [8, 9, 10],
    "eta": [.1],
    "min_child_weight": [0],
    "gamma": [0.15, 0.2, 0.25],
    "seed": [42]
}
xgBoost_SearchSpace = {
    'learning_rate': [0.01, 0.1, 0.2],            # Step size shrinkage used to prevent overfitting
    'n_estimators': [50, 100, 200],               # Number of boosting rounds
    'max_depth': [7, 10, 15],                       # Maximum depth of a tree
    'min_child_weight': [1] ,                # Minimum sum of instance weight needed in a child                # Subsample ratio of the training instances
    'gamma': [0, 0.1, 0.2],                       # Minimum loss reduction required to make a further partition on a leaf node
}

rf_SearchSpace = {
    'n_estimators': [50, 70, 100],          
    'max_depth': [17, 19, 20],           
    'min_samples_split': [2],          
    'min_samples_leaf': [1],           
    'max_features': ['log2'],
    'criterion': ["squared_error"],
    'max_samples': [0.8, 0.9, 1],
    'ccp_alpha': [2.4, 2.5, 2.6]

}

svr_SearchSpace = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1],
    'epsilon': [0.1, 0.2, 0.5]
}

lasso_SearchSpace = {
    'alpha': [0.05, 0.1, 0.15],  # Different values for alpha
    'tol': [.05, .1, .15],  # Different values for tolerance
    'max_iter': [1000, 2000, 2500],  # Different values for maximum iterations
    'selection': ['random'],
    'precompute': [False],
    'positive': [False]
}


scoring = {
    'r2': 'r2',  
    'neg_mean_squared_error': 'neg_mean_squared_error'
}

model = xgb.XGBRFRegressor()


In [107]:
GS = GridSearchCV(estimator=SVR(),
                  param_grid= svr_SearchSpace,
                  scoring = scoring,
                  refit='r2',
                  cv=5,
                  verbose=4)

In [108]:
GS.fit(X, y)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV 1/5] END C=0.1, epsilon=0.1, gamma=scale, kernel=linear; neg_mean_squared_error: (test=-92.251) r2: (test=0.070) total time=   0.0s
[CV 2/5] END C=0.1, epsilon=0.1, gamma=scale, kernel=linear; neg_mean_squared_error: (test=-114.775) r2: (test=0.460) total time=   0.0s
[CV 3/5] END C=0.1, epsilon=0.1, gamma=scale, kernel=linear; neg_mean_squared_error: (test=-33.988) r2: (test=0.885) total time=   0.1s
[CV 4/5] END C=0.1, epsilon=0.1, gamma=scale, kernel=linear; neg_mean_squared_error: (test=-8.164) r2: (test=0.916) total time=   0.0s
[CV 5/5] END C=0.1, epsilon=0.1, gamma=scale, kernel=linear; neg_mean_squared_error: (test=-3.559) r2: (test=0.985) total time=   0.0s
[CV 1/5] END C=0.1, epsilon=0.1, gamma=scale, kernel=rbf; neg_mean_squared_error: (test=-273.323) r2: (test=-1.755) total time=   0.0s
[CV 2/5] END C=0.1, epsilon=0.1, gamma=scale, kernel=rbf; neg_mean_squared_error: (test=-259.795) r2: (test=-0.223) total t

In [102]:
GS.best_params_

{'alpha': 0.1,
 'max_iter': 2000,
 'positive': False,
 'precompute': False,
 'selection': 'random',
 'tol': 0.05}

In [103]:
GS.best_score_

0.6994697278267903

In [104]:
GS.best_estimator_

Lasso(alpha=0.1, max_iter=2000, selection='random', tol=0.05)

In [10]:
ndf = pd.DataFrame(GS.cv_results_)
ndf.to_csv("cv_results.csv")